In [1]:
#install packages
!pip install --quiet pandas numpy scipy scikit-learn plotly seaborn


In [2]:
#stimulated dataset
import numpy as np
import pandas as pd

# Simulate OTU table (samples × taxa)
n_samples, n_taxa = 50, 20
np.random.seed(42)
otu = np.random.poisson(lam=50, size=(n_samples, n_taxa))
otu = pd.DataFrame(otu, columns=[f"Taxon_{i+1}" for i in range(n_taxa)])
otu.index = [f"S{i+1}" for i in range(n_samples)]

# Metadata (BMI, age, sex, obesity group)
meta = pd.DataFrame({
    "BMI": np.round(np.random.normal(27, 5, n_samples), 1),
    "age": np.random.randint(18, 70, n_samples),
    "sex": np.random.choice(["M", "F"], n_samples)
}, index=otu.index)
meta["obesity"] = pd.cut(meta["BMI"], bins=[0,25,30,100], labels=["normal","overweight","obese"])

otu.head(), meta.head()


(    Taxon_1  Taxon_2  Taxon_3  Taxon_4  Taxon_5  Taxon_6  Taxon_7  Taxon_8  \
 S1       47       55       42       52       58       43       46       49   
 S2       64       52       38       33       48       58       45       41   
 S3       50       46       61       48       44       45       63       53   
 S4       34       61       41       54       44       47       53       39   
 S5       47       60       53       52       39       51       53       55   
 
     Taxon_9  Taxon_10  Taxon_11  Taxon_12  Taxon_13  Taxon_14  Taxon_15  \
 S1       52        45        49        43        52        52        46   
 S2       56        54        56        47        59        46        46   
 S3       57        60        46        44        48        40        65   
 S4       46        54        50        53        54        64        47   
 S5       46        53        53        39        45        60        57   
 
     Taxon_16  Taxon_17  Taxon_18  Taxon_19  Taxon_20  
 S1       

In [3]:
#cleaning the  dataset
# Remove low-depth samples & rare taxa
sample_sums = otu.sum(axis=1)
otu = otu[sample_sums >= 500]   # keep good samples

taxa_sums = otu.sum(axis=0)
otu = otu.loc[:, taxa_sums >= 100]   # keep common taxa
otu.shape


(50, 20)

In [4]:
#analysis
def shannon(counts):
    p = counts / counts.sum()
    p = p[p > 0]
    return -(p * np.log(p)).sum()

alpha = otu.apply(shannon, axis=1)
alpha = alpha.to_frame("shannon").join(meta)
alpha.head()


,shannon,BMI,age,sex,obesity
S1,2.991051,29.7,38,M,overweight
S2,2.982462,27.0,28,M,overweight
S3,2.984521,24.8,56,M,normal
S4,2.983140,26.5,45,M,overweight
S5,2.987674,26.6,22,F,overweight


In [5]:
#pca on transformed data
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

rel_abund = otu.div(otu.sum(axis=1), axis=0)
clr = np.log(rel_abund + 1e-6) - np.log(rel_abund + 1e-6).mean(axis=1).values[:,None]

pca = PCA(n_components=2)
pcs = pca.fit_transform(clr)
pcoa_df = pd.DataFrame(pcs, columns=["PC1","PC2"], index=otu.index).join(meta)


In [6]:
#visualization
import plotly.express as px

# PCA scatter
fig = px.scatter(pcoa_df, x="PC1", y="PC2", color="obesity", hover_data=["BMI","age","sex"],
                 title="Microbiome PCA (CLR transformed)")
fig.show()

# Alpha diversity boxplot
fig2 = px.box(alpha.reset_index(), x="obesity", y="shannon", points="all",
              title="Shannon Diversity by Obesity Status")
fig2.show()


In [7]:
#saving resut
otu.to_csv("otu_clean.csv")
meta.to_csv("metadata.csv")
alpha.to_csv("alpha_diversity.csv")
pcoa_df.to_csv("ordination.csv")
